In [28]:
import pandas as pd
from pulp import *
PATH = 'data/'

current_df = pd.read_csv(PATH + '2021-22/gws/merged_gw.csv', index_col=False)
gw_35 = current_df[current_df.GW == 35]

GW_dict = {}
GWs = current_df.GW.unique()
              
# Solving for the best teams from each week until end
for GW in sorted(GWs)[:1]:
    player_price_dict = {} # key: player, value: (name, team, position, price, total points)
    players = current_df[current_df.GW == GW].name.unique()
    # print("Number players in week {} is {}".format(GW, len(players)))
    for i, player in enumerate(players):
        player_entries = current_df[current_df.name == player]
        total_points = player_entries[player_entries.GW >= GW]['total_points'].sum()
        player_team = player_entries[player_entries.GW == GW]['team'].values[0]
        player_price = player_entries[player_entries.GW == GW]['value'].values[0]
        player_position = player_entries[player_entries.GW == GW]['position'].values[0]
        player_name = player_entries[player_entries.GW == GW]['name']
        if player not in player_price_dict:
                val = (player,player_team, player_position, player_price, total_points)
                player_price_dict[player] = val             
    GW_dict[GW] = player_price_dict
        #print("Player {} had {} price total in all {} played.".format(player, player_price, len(player_entries)))

gw_35.head()
data = gw_35[['name', 'team', 'position', 'total_points', 'value']]        
data.head()
        
# Helper variables
POS = data.position.unique()
CLUBS = data.team.unique()

BUDGET = 1000
pos_available = {
'DEF': 5,
'FWD': 3,
'MID': 5,
'GK': 2,
}

# Initialize Variables
solver_dict = {}
points_dict = {}
for GW in GWs[:1]:
    # print(GW)
    cur_player_price_dict = GW_dict[GW]
    names, teams, positions, prices, points = list(zip(*cur_player_price_dict.values()))
    players = [LpVariable("player_" + str(i), cat="Binary") for i in range(len(names))]
    #print(players)
    # Initialize the problem
    prob = LpProblem("FPL_Player_Choices", LpMaximize)
    # Define the objective
    prob += lpSum(players[i] * points[i] for i in range(len(names))) # Objective
    # Build the constraints
    prob += lpSum(players[i] * prices[i] for i in range(len(names))) <= BUDGET # Budget Limit

    for pos in POS:
        prob += lpSum(players[i] for i in range(len(names)) if positions[i] == pos) <= pos_available[pos] # Position Limit

    for club in CLUBS:
        prob += lpSum(players[i] for i in range(len(names)) if teams[i] == club) <= 3 # Club Limit
        
    # Solve the problem
    prob.solve()
    for v in prob.variables():
        if v.varValue != 0:
            index = int(v.name.split("_")[1])
            name = names[index]
            club = teams[index]
            position = positions[index]
            point = points[index]
            price = prices[index]
            # print(name, position, club, point, price, sep=" | ")
            score = str(prob.objective)
            constraint = [str(const) for const in prob.constraints.values()][0]
            solver_dict[name] = name, position, club, point, price
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraint = constraint.replace(v.name, str(v.varValue))

    score_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', score) )
    constraint_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', constraint) )

    # print(value(prob.objective))
    points_dict[GW] = value(prob.objective)
    
    
print(points_dict)
print(solver_dict)

# names, positions, clubs, points, prices = list(zip(*solver_dict.values()))
# totals_dict = {}
# my_team = {}
# for i in range(len(names)):
#     for GW in sorted(GWs)[:35]:
#         player_entries = current_df[current_df.name == names[i]]
#         points = player_entries[player_entries.GW == GW]['total_points'].sum()
#         totals_dict[GW] = points
#     my_team[names[i]] = positions[i],totals_dict
# print(my_team)




Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kylecasey/Documents/Fantasy-Premier-League/venv/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/12c609891bfc4c469ecb5ae493dada42-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/3s/js3l59qs4q73dcnvx8spqhq40000gn/T/12c609891bfc4c469ecb5ae493dada42-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 3248 RHS
At line 3274 BOUNDS
At line 3828 ENDATA
Problem MODEL has 25 rows, 553 columns and 1659 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2477.67 - 0.00 seconds
Cgl0004I processed model has 25 rows, 450 columns (450 integer (448 of which binary)) and 1350 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 4 integers unsatisfied sum - 1.33333
